# Tabular Prediction

## Tập dữ liệu Dự đoán quyết định chơi tennis

Tập dữ liệu này chứa thông tin về các đặc trưng thời tiết liên quan đến quyết định chơi tennis (PlayTennis). Bộ dữ liệu gồm 4 thuộc tính:
- `Outlook`: trạng thái thời tiết (Sunny, Overcast, Rain)
- `Temperature`: nhiệt độ (Hot, Mild, Cool)
- `Humidity`: độ ẩm (High, Normal)
- `Wind`: gió (Weak, Strong)
- `PlayTennis`: quyết định chơi tennis (Yes, No)

Nhiệm vụ: Trả lời các câu hỏi liên quan đến quá trình xây dựng cây quyết định từ **10 dòng dữ liệu đầu tiên** và sử dụng mô hình cây đó để dự đoán và kiểm tra trên **4 dòng dữ liệu còn lại**.

**Bộ dữ liệu để xây dựng cây quyết định:**

| Outlook | Temperature | Humidity | Wind | PlayTennis |
|-----------|-------------|----------|--------|------------|
| Sunny | Hot | High | Weak | No |
| Sunny | Hot | High | Strong | No |
| Overcast | Hot | High | Weak | Yes |
| Rain | Mild | High | Weak | Yes |
| Rain | Cool | Normal | Weak | Yes |
| Rain | Cool | Normal | Strong | No |
| Overcast | Cool | Normal | Strong | Yes |
| Sunny | Mild | High | Weak | No |
| Sunny | Cool | Normal | Weak | Yes |
| Rain | Mild | Normal | Weak | Yes |

**Bộ dữ liệu kiểm tra:**

| Outlook | Temperature | Humidity | Wind | PlayTennis |
|-----------|-------------|----------|--------|------------|
| Sunny | Mild | Normal | Strong | Yes |
| Overcast | Mild | High | Strong | Yes |
| Overcast | Hot | Normal | Weak | Yes |
| Rain | Mild | High | Strong | No |


In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from math import log2

Bộ dữ liệu ban đầu

In [2]:
data = pd.read_csv('../../Datasets/Decision Tree/tennis.csv')
data

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes
5,Rainy,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainy,Mild,Normal,Weak,Yes


Xác định đặc trưng và nhãn của dữ liệu

In [3]:
X = data.iloc[:, :-1].values  # All columns except the last one are features
y = data.iloc[:, -1].values  # The last column is the label
feature_names = data.columns[:-1].tolist()
print(f"Các đặc trưng: {feature_names}")

Các đặc trưng: ['Outlook', 'Temperature', 'Humidity', 'Wind']


Tách dữ liệu thành 2 phần:
- Dữ liệu để xây dựng cây quyết định (10 dòng đầu tiên)
- Dữ liệu kiểm tra (4 dòng còn lại)

In [4]:
X_train, y_train = X[:10], y[:10]
X_test, y_test = X[10:], y[10:]

Các bước xây dựng cây quyết định (khi sử dụng Entropy):

1. Bắt đầu với toàn bộ tập dữ liệu
2. Chọn thuộc tính chia nhánh tốt nhất dựa trên độ đo Entropy và Information Gain
3. Chia dữ liệu thành các nhánh con dựa trên thuộc tính đó
4. Lặp lại quá trình đến khi:
    - Tất cả các mẫu thuộc cùng một lớp
    - Không còn thuộc tính nào để chia nhánh
    - Đạt đến điều kiện dừng (số lượng mẫu nhỏ nhất hoặc độ sâu tối đa)

#### **Câu 1:** Entropy của tập dữ liệu ban đầu là bao nhiêu (làm tròn đến 4 chữ số thập phân)?

A. 0.7219  
B. 0.9710  
C. 0.8113  
D. 0.9183

Đáp án: B

In [5]:
def calculate_entropy(y):
    # Count the number of occurrences of each label
    label_counts = Counter(y)
    # Calculate the frequency of each label
    label_freqs = np.array(list(label_counts.values())) / len(y)
    # Calculate entropy
    entropy = -np.sum(label_freqs * np.log2(label_freqs))
    return entropy

In [6]:
entropy_root = calculate_entropy(y_train)
print(f"Entropy của tập dữ liệu ban đầu: {entropy_root:.4f}")

Entropy của tập dữ liệu ban đầu: 0.9710


#### **Câu 2:** Với tập dữ liệu huấn luyện ban đầu, Information Gain của thuộc tính Temperature là bao nhiêu (làm tròn đến 4 chữ số thập phân)?

A. 0.0913   
B. 0.0955   
C. 0.8113   
D. 0.9183

Đáp án: B

Ta sẽ tính Entropy của tập dữ liệu nếu chia theo thuộc tính Temperature, sau đó Information Gain sẽ được tính bằng:

$IG(Temperature) = Entropy(S) - Entropy(S, Temperature)$

In [7]:
column = "Temperature"
col_idx = feature_names.index(column) # Index of the column corresponding to the feature

# Unique values & its count in 'Temperature' column
values, count = np.unique(X_train[:, col_idx], return_counts=True)

# Calculate the entropy of each partition
for value in values:
    # Mask to filter rows that have the value of the feature == the unique value
    mask = X_train[:, col_idx] == value
    print(f"Entropy khi {column} = {value}: {calculate_entropy(y_train[mask]):.4f}")

# Calculate the sum of the entropy of each partition
entropy_temperature = np.sum(
    [
        count[i] / len(y_train) * calculate_entropy(y_train[X_train[:, col_idx] == values[i]])
        for i in range(len(values))
    ]
)
information_gain = entropy_root - entropy_temperature
print(f"Information gain khi chia theo {column}: {information_gain:.4f}")

Entropy khi Temperature = Cool: 0.8113
Entropy khi Temperature = Hot: 0.9183
Entropy khi Temperature = Mild: 0.9183
Information gain khi chia theo Temperature: 0.0955


#### **Câu 3:** Thuộc tính nào sẽ được chọn để chia nhánh đầu tiên?

A. Outlook  
B. Temperature  
C. Humidity     
D. Wind

Đáp án: A

Tương tự như câu 2, ta sẽ tính Information Gain của các thuộc tính Outlook, Temperature, Humidity, Wind và chọn thuộc tính có Information Gain lớn nhất.

In [8]:
features = feature_names
information_gains = []
for column in features:
    col_idx = feature_names.index(column)
    values, count = np.unique(X_train[:, col_idx], return_counts=True)
    entropy_feature = np.sum(
        [
            count[i] / len(y_train) * calculate_entropy(y_train[X_train[:, col_idx] == values[i]])
            for i in range(len(values))
        ]
    )
    information_gain = entropy_root - entropy_feature
    information_gains.append(information_gain)
    print(f"Information gain khi chia theo {column}: {information_gain:.4f}")

best_feature_idx = np.argmax(information_gains)
best_feature = features[best_feature_idx]
print(f"Đặc trưng tốt nhất để chia: {best_feature}")

Information gain khi chia theo Outlook: 0.3219
Information gain khi chia theo Temperature: 0.0955
Information gain khi chia theo Humidity: 0.1245
Information gain khi chia theo Wind: 0.0913
Đặc trưng tốt nhất để chia: Outlook


#### Code xây dựng hoàn chỉnh cây

Ta tiếp tục lặp lại các bước từ câu 1-3 để xây dựng cây quyết định hoàn chỉnh từ dữ liệu huấn luyện.

Dưới đây là code Python hoàn chỉnh để xây dựng cây quyết định từ dữ liệu huấn luyện và dự đoán trên dữ liệu kiểm tra.


In [9]:
# Define the DecisionTree class
class DecisionTree:
    def __init__(self, feature_names):
        self.tree = None
        self.feature_names = feature_names

    def entropy(self, y, node_name=""):
        freq = Counter(y)
        total = len(y)
        entropy_value = -sum(
            (count / total) * log2(count / total) for count in freq.values()
        )
        entropy_value = 0 if entropy_value == -0.0 else entropy_value
        if node_name:
            print(
                f"Node: {node_name} | Label Distribution: {dict(freq)} | Entropy: {entropy_value:.4f}"
            )
        return entropy_value

    # Calculate the information gain
    def info_gain(self, X_column, y, node_name):
        total_entropy = self.entropy(y, node_name)
        values, counts = np.unique(X_column, return_counts=True)
        weighted_entropy = sum(
            (counts[i] / sum(counts))
            * self.entropy(y[X_column == values[i]], f"{values[i]}")
            for i in range(len(values))
        )
        info_gain_value = total_entropy - weighted_entropy
        return info_gain_value

    # Determine the best feature to split on based on information gain
    def best_feature_split(self, X, y):
        best_gain = -1
        best_feature = None
        print("\n=== Information Gain for Each Feature ===\n")
        for col in range(X.shape[1]):
            gain = self.info_gain(X[:, col], y, node_name="Total" if col == 0 else "")
            print(f"Feature {self.feature_names[col]}: Information Gain = {gain:.4f}\n")
            if gain > best_gain:
                best_gain = gain
                best_feature = col
        print(
            f"Best feature to split on {self.feature_names[best_feature]} with Information Gain: {best_gain:.4f}\n"
        )
        return best_feature

    # Fit the model and build the tree
    def fit(self, X, y, feature_names, depth=0):

        print(f"\n--- Building Tree at Depth {depth} ---")
        if len(set(y)) == 1:
            print(f"Stopping condition met at depth {depth}, class: {y[0]}")
            return y[0]

        if X.shape[1] == 0:
            most_common = Counter(y).most_common(1)[0][0]
            print(
                f"No more features to split on at depth {depth}, returning majority class: {most_common}"
            )
            return most_common

        feature_idx = self.best_feature_split(X, y)
        best_feature_name = feature_names[feature_idx]
        tree = {best_feature_name: {}}
        print(f"=> Splitting on feature {best_feature_name} at depth {depth}\n")

        # Split the dataset based on the best feature
        for value in np.unique(X[:, feature_idx]):
            sub_X = X[X[:, feature_idx] == value]
            sub_y = y[X[:, feature_idx] == value]
            print()
            print(f"="*45)
            print(f"|| Subtree for feature {best_feature_name} = {value} ||")
            print(f"="*45)
            sub_X = np.delete(sub_X, feature_idx, axis=1)
            new_feature_names = (
                feature_names[:feature_idx] + feature_names[feature_idx + 1 :]
            )
            tree[best_feature_name][value] = self.fit(
                sub_X, sub_y, new_feature_names, depth + 1
            )

        self.tree = tree
        return tree

    # Predict the class labels for the samples
    def predict(self, X):
        def classify(tree, sample):
            if not isinstance(tree, dict):
                return tree
            feature = list(tree.keys())[0]
            feature_idx = self.feature_names.index(feature)
            value = sample[feature_idx]

            # Check if the feature value is missing ("")
            if value == "":
                print(
                    f"Feature '{feature}' is missing in the sample so the model can't predict"
                )

            # Find the subtree or return None if the feature is required and not found
            subtree = tree[feature].get(value, None)
            if subtree is None:
                return None  # If we can't find the path, return None

            if isinstance(subtree, dict):
                return classify(subtree, sample)
            else:
                return subtree

        # Apply classification for all samples in X
        return (
            np.array([classify(self.tree, x) for x in X])
            if len(X.shape) > 1
            else classify(self.tree, X)
        )

    # Function to print the tree
    def visualize(self):
        def print_tree(node_name, node_values, depth):
            for value in node_values:
                if isinstance(node_values[value], dict):
                    print(f"{'|  ' * depth}{node_name} = {value}")
                    sub_tree = node_values[value]
                    new_node = list(sub_tree.keys())[0]
                    print_tree(new_node, sub_tree[new_node], depth + 1)
                else:
                    print(f"{'|  ' * depth}{node_name} = {value}: {node_values[value]}")

        if self.tree:
            root_node = list(self.tree.keys())[0]
            print("\n--- Decision Tree ---")
            print_tree(root_node, self.tree[root_node], 0)

In [10]:
# Initialize and fit the decision tree
tree = DecisionTree(feature_names)
tree.fit(X_train, y_train, feature_names)
tree.visualize()


--- Building Tree at Depth 0 ---

=== Information Gain for Each Feature ===

Node: Total | Label Distribution: {'No': 4, 'Yes': 6} | Entropy: 0.9710
Node: Overcast | Label Distribution: {'Yes': 2} | Entropy: 0.0000
Node: Rainy | Label Distribution: {'Yes': 3, 'No': 1} | Entropy: 0.8113
Node: Sunny | Label Distribution: {'No': 3, 'Yes': 1} | Entropy: 0.8113
Feature Outlook: Information Gain = 0.3219

Node: Cool | Label Distribution: {'Yes': 3, 'No': 1} | Entropy: 0.8113
Node: Hot | Label Distribution: {'No': 2, 'Yes': 1} | Entropy: 0.9183
Node: Mild | Label Distribution: {'Yes': 2, 'No': 1} | Entropy: 0.9183
Feature Temperature: Information Gain = 0.0955

Node: High | Label Distribution: {'No': 3, 'Yes': 2} | Entropy: 0.9710
Node: Normal | Label Distribution: {'Yes': 4, 'No': 1} | Entropy: 0.7219
Feature Humidity: Information Gain = 0.1245

Node: Strong | Label Distribution: {'No': 2, 'Yes': 1} | Entropy: 0.9183
Node: Weak | Label Distribution: {'No': 2, 'Yes': 5} | Entropy: 0.8631
Fea

#### **Câu 4:** Cây quyết định hoàn chỉnh sẽ có bao nhiêu lá?

A. 4    
B. 5    
C. 6    
D. 7    

Đáp án: C

Kết quả cây quyết định:

=> Có tổng cộng 6 nút lá

<img src="https://i.imgur.com/chTFTY6.png" alt="DT_result" width="800"/>

#### **Câu 5:** Dùng cây quyết định đã xây dựng trên dữ liệu huấn luyện, dự đoán kết quả của dòng dữ liệu cuối cùng trong tập kiểm tra.

A. Strong   
B. Cool     
C. Yes  
D. No

Đáp án: D

In [11]:
# Use the model to make predictions
tree.predict(X_test[-1])

'No'

#### **Câu 6:** Dùng cây quyết định đã xây dựng, dự đoán kết quả cho mẫu dữ liệu sau: Outlook = Overcast, Wind = Weak.

A. Yes  
B. No   
C. Thiếu giá trị của thuộc tính Temperature nên không dự đoán được     
D. Thiếu giá trị của thuộc tính Humidity nên không dự đoán được

Đáp án: A

In [12]:
# Sample 1
sample_1 = np.array(["Overcast", "", "", "Weak"])
tree.predict(sample_1)

'Yes'

#### **Câu 7:** Dùng cây quyết định đã xây dựng, dự đoán kết quả cho mẫu dữ liệu sau: Temperature = Hot, Outlook = Rainy.

A. Yes  
B. No   
C. Thiếu giá trị của thuộc tính Wind nên không dự đoán được  
D. Thiếu giá trị của thuộc tính Humidity nên không dự đoán được  

Đáp án: C

In [13]:
# Sample 2
sample_2 = np.array(["Rainy", "Hot", "", ""])
tree.predict(sample_2)

Feature 'Wind' is missing in the sample so the model can't predict


Vì thiếu thông tin về thuộc tính Wind, ta không thể dự đoán kết quả chính xác cho mẫu dữ liệu này.

#### **Câu 8:** Độ chính xác của mô hình cây trên tập kiểm tra là bao nhiêu?

A. 50%  
B. 75%  
C. 100%     
D. 25%

Đáp án: B

In [14]:
# Evaluate the model using the last 4 rows for testing
y_test_pred = tree.predict(X_test)
print(f"Predict test set: {y_test_pred}")
print(f"Actual test set: {y_test}")

accuracy = np.mean(y_test_pred == y_test)
print(f"Accuracy: {accuracy:.2f}")

Predict test set: ['No' 'Yes' 'Yes' 'No']
Actual test set: ['Yes' 'Yes' 'Yes' 'No']
Accuracy: 0.75


#### Câu 9: Nếu một thuộc tính liên tục được chọn làm thuộc tính phân chia, bước tiếp theo sẽ là gì?

A. Chuyển đổi thuộc tính liên tục thành thuộc tính phân loại

B. Chọn điểm cắt để chia thuộc tính thành hai miền

C. Bỏ qua thuộc tính liên tục đó

D. Chia thuộc tính thành nhiều phân khúc dựa trên các giá trị trung bình

Đáp án: B.

#### Câu 10: Một thuộc tính có giá trị Information Gain bằng 0 cho thấy điều gì?
A. Thuộc tính không mang lại thông tin để phân biệt các lớp

B. Thuộc tính là quan trọng nhất

C. Thuộc tính này có thể loại bỏ trong quá trình huấn luyện

D. Thuộc tính gây overfitting

Đáp án: A.